# Learning and Decision Making

## Laboratory 3: Partially observable Markov decision problems

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. Modeling

Consider once again the princess saving problem described in the Homework and which you described as a POMDP.

Recall that:

* The princess can be in any one of two towers: Tower *A* and Tower *B*. The knight must decide which tower do invade to rescue the princess. 

* The knight can try to _peer_ at the towers, to figure out where the princess may be. 

* When the agent peers, it sees the princess in the right location with a probability of 0.9, and at the wrong location with probability 0.1.

* Whenever the knight invades a tower, the "world" resets.

Consider throughout that $\gamma=0.9$.

---

#### Activity 1.        

Implement your POMDP in Python. In particular,

* Create a list with all the states;
* Create a list with all the actions;
* Create a list with all the observations
* For each action, define a `numpy` array with the corresponding transition probabilities;
* For each action, define a `numpy` array with the corresponding observation probabilities;
* Define a `numpy` array with the cost describing the problem. Make sure that the costs lie in the interval [0,1] and that the cost for peering is in the middle between the cost for saving and being captured.

The order for the states and actions used in the transition probability and cost matrices should match that in the lists of states and actions. 

**Note**: Don't forget to import `numpy`.

---

In [4]:
import numpy as np

states = ['A', 'B']
actions = ['a', 'b', 'p']
obs = ['A', 'B', 'N']

Pa = Pb = np.array([[0.5, 0.5],
                    [0.5, 0.5]])

Pp = np.array([[1.0, 0.0],
               [0.0, 1.0]])

P_dict = {'a': Pa, 'b': Pb, 'p': Pp}

Oa = Ob = np.array([[0.0, 0.0, 1.0],
                    [0.0, 0.0, 1.0]])

Op = np.array([[0.9, 0.1, 0.0],
               [0.1, 0.9, 0.0]])

C = np.array([[0.0, 1.0, 0.5],
              [1.0, 0.0, 0.5]]) 

O_dict = {'a': Oa, 'b': Ob, 'p': Op}



### 2. Sampling

You are now going to sample random trajectories of your POMDP and observe the impact it has on the corresponding belief.

---

#### Activity 2.

Generate a random POMDP trajectory using a uniformly random policy. In particular, from a random initial state $x_0$ generate:

1. A sequence of 10,000 states by selecting the actions uniformly at random;
2. The corresponding sequence of 10,000 actions;
3. The corresponding sequence of 10,000 observations.

---

In [114]:
import random

sequence_states = np.zeros(10000, dtype=str)
sequence_actions = np.zeros(10000, dtype=str)
sequence_observations = np.zeros(10000, dtype=str)

sequence_states[0] = random.choice(states)
sequence_actions[0] = random.choice(actions)

if sequence_actions[0] == 'a' or sequence_actions[0] == 'b':
    sequence_observations[0] = 'N'
else:
    sequence_observations[0] = np.random.choice(obs, 1, p = Op[0 if sequence_states[0] == 'A' else 1].tolist())[0]

for i in range(1,10000):
    if sequence_actions[i-1] == 'p':
        sequence_states[i] = sequence_states[i-1]
    else:
        sequence_states[i] = np.random.choice(states, 1, p = Pa[0 if sequence_states[i-1] == 'A' else 1].tolist())[0]
    
    sequence_actions[i] = random.choice(actions)
    
    if sequence_actions[i] == 'a' or sequence_actions[i] == 'b':
        sequence_observations[i] = 'N'
    else:
        sequence_observations[i] = np.random.choice(obs, 1, p = Op[0 if sequence_states[i] == 'A' else 1].tolist())[0]

---

#### Activity 3.

For the POMDP trajectory generated in Activity 2, compute the corresponding sequence of beliefs, assuming that the initial belief is $\mathbf{b}_0=[0.5, 0.5]$. Report the resulting beliefs, ignoring duplicate beliefs or beliefs whose distance is smaller than $10^{-3}$.

**Note 1:** You may want to define a function `belief_update` that receives a belief, an action and an observation and returns the updated belief.

**Note 2:** To compute the distance between vectors, you may find useful `numpy`'s function `linalg.norm`.


---

In [115]:
from sklearn.preprocessing import normalize

obs_col_dict = {'A': 0, 'B': 1, 'N': 2}

init_belief = np.array([[0.5, 0.5]])

sequence_beliefs = np.array(init_belief)

def belief_update(belief, action, observation):
    aux1 = belief.dot(P_dict[action])
    aux2 = aux1.dot(np.diag(O_dict[action][:,obs_col_dict[observation]]))
    new_belief = normalize(aux2.reshape(1,2), norm='l1')
    return new_belief

'''beliefs estão relacionados com a ação ou com o valor?'''
for i in range(10000):
    flag = False
    new_belief = belief_update(sequence_beliefs[-1], sequence_actions[i], sequence_observations[i])
    for old_belief in sequence_beliefs:
        if np.linalg.norm(old_belief-new_belief) < 10e-3:
            flag = True
            break 
    if new_belief not in sequence_beliefs and not flag:
        sequence_beliefs = np.append(sequence_beliefs, new_belief, axis=0)
print(sequence_beliefs)

[[0.5        0.5       ]
 [0.1        0.9       ]
 [0.01219512 0.98780488]
 [0.00136986 0.99863014]]


### 3. Solution methods

In this section you are going to compare different non-exact solution methods.

---

#### Activity 4

Compute the solution for the underlying MDP and report the corresponding optimal policy and optimal cost-to-go. 

** Note:** You may reuse code from previous labs.

---

In [128]:
'''P = np.squeeze(np.asarray(m2))
J = np.zeros((6,1))
print(P)
gamma = Y
err = 1
i = 0

initial_time = time.time()

while err > 1e-8:
    QUp = C + Y*PUp.dot(J)
    QDown = C + Y*PDown.dot(J)
    QLeft = C + Y*PLeft.dot(J)
    QRight = C + Y*PRight.dot(J)
    Jnew = np.min((QUp, QDown, QLeft, QRight), axis=0)
    err = np.linalg.norm(Jnew - J)
    i += 1
    J = Jnew
J_final = J
print(J_final)
print("Number of interations:",i)

print("Time:", time.time() - initial_time)'''

###################################################################################################################

pi = np.ones((2,3))

quit = False

#initial_time = time.time()

Y = 0.9

while not quit:
    cpi = np.diag(pi[:,0]).dot(C) + np.diag(pi[:,1]).dot(C) + np.diag(pi[:,2]).dot(C)
    Ppi = np.diag(pi[:,0]).dot(Pa) + np.diag(pi[:,1]).dot(Pb) + np.diag(pi[:,2]).dot(Pp)
    J = np.linalg.inv(np.eye(2) - Y*Ppi).dot(cpi)
    
    Qa = np.transpose(C[:,0].reshape((1,2))) + Y*Pa.dot(J),
    Qb = np.transpose(C[:,1].reshape((1,2))) + Y*Pb.dot(J)
    Qp = np.transpose(C[:,2].reshape((1,2))) + Y*Pp.dot(J)
    
    pinew = np.zeros((2,3))
    
    pinew[:, 0, None] = np.isclose(Qa, np.min([Qa, Qb, Qp], axis=0), atol=1e-10, rtol=1e-10).astype(int)
    pinew[:, 1, None] = np.isclose(Qb, np.min([Qa, Qb, Qp], axis=0), atol=1e-10, rtol=1e-10).astype(int)
    pinew[:, 2, None] = np.isclose(Qp, np.min([Qa, Qb, Qp], axis=0), atol=1e-10, rtol=1e-10).astype(int)
    
    pinew = pinew / np.sum(pinew, axis=1, keepdims = True)
    
    quit = (pi == pinew).all()
    pi = pinew

print(pi)
#print("Number of iterations:", it)
#print("Time:", time.time() - initial_time)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

---

#### Activity 5

For each of the beliefs computed in Activity 3, compute the action prescribed by:

* The MLS heuristic;
* The AV heuristic;
* The Q-MDP heuristic.

---

In [ ]:
# Insert your code here.

---

#### Activity 6

Suppose that the optimal cost-to-go function for the POMDP can be represented using the $\alpha$-vectors

$$
\left\{
\begin{bmatrix}
2.795\\
3.795
\end{bmatrix},
\begin{bmatrix}
3.795\\
2.795
\end{bmatrix},
\begin{bmatrix}
3.105\\
3.105
\end{bmatrix}\right\}$$

corresponding to the actions 'Invade Tower A', 'Invade Tower B' and 'Peer', respectively. Represent the optimal cost-to-go function and compare the optimal policy with the MDP heuristics from Activity 5 in the beliefs computed in Activity 3.

** Note: ** Don't forget to import `matplotlib`, and use the magic `%matplotlib notebook`.

---

In [ ]:
# Insert your code here.